In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
vacs_db0 = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv")
vacs_db0.head()

In [ ]:
cntry_df= pd.read_csv("/kaggle/input/undata-country-profiles/country_profile_variables.csv")
#cntry_df.head()
#cntry_df.info()
#cntry_df.describe()
cntry_df.isnull().sum()
cntry_df = cntry_df.rename(columns={'GDP: Gross domestic product (million current US$)':'GDP (million in US$)', 'Population in thousands (2017)' :'Population in thousands'})
country_profile=cntry_df[['country', 'Region', 'Surface area (km2)','Population in thousands','GDP (million in US$)','GDP per capita (current US$)','Life expectancy at birth (females/males, years)','Population age distribution (0-14 / 60+ years, %)','Health: Total expenditure (% of GDP)','Health: Physicians (per 1000 pop.)','Seats held by women in national parliaments %','Mobile-cellular subscriptions (per 100 inhabitants)']].copy()
#country_profile.info()
#group the covid dataset by country 
#g_covid_df = country_profile.groupby(by=['country','iso_code'], sort=False,as_index = False)['total_vaccinations'].max()

In [ ]:
country_profile.describe()
country_profile.info()

In [ ]:
#check for outliers
qLow = country_profile.quantile(0.01)
qLow
#Fix the ouliers
qLow = country_profile['GDP (million in US$)'].quantile(0.01)
country_profile['GDP (million in US$)']= np.where(country_profile['GDP (million in US$)'] == qLow,0,country_profile['GDP (million in US$)'])



qLow = country_profile['Health: Total expenditure (% of GDP)'].quantile(0.01)
qLow = country_profile['Health: Total expenditure (% of GDP)']= np.where(country_profile['Health: Total expenditure (% of GDP)'] == qLow, 0,country_profile['Health: Total expenditure (% of GDP)'])

In [ ]:
#group the covid dataset by country 
g_covid_df = vacs_db0.groupby(by=['country','iso_code'], sort=False,as_index = False)['total_vaccinations'].max()
g_covid_df.head()


In [ ]:
#replace country name for USA as it is not same in both dataset
g_covid_df.country=g_covid_df.country.replace({'United States':'United States of America'})

#merge total vaccinations column with country profile data
m_df = country_profile.merge(g_covid_df['total_vaccinations'], left_on=country_profile['country'], right_on=g_covid_df['country']).drop(columns=['key_0'])

m_df.corr()
m_df.head()
m_df.info()
#m_df['GDP per capita (current US$)']



In [ ]:
m_df['Region']
m_df['total_vaccinations']

m_df_region=m_df.groupby(by=['Region'], sort=False,as_index = False)['total_vaccinations'].mean()
m_df_region_total=m_df.groupby(by=['Region'], sort=False,as_index = False)['total_vaccinations'].sum()


In [ ]:
fig,ax=plt.subplots(figsize=(30,5))
ax.plot(m_df_region['Region'],m_df_region['total_vaccinations'],color='black',label='mean')
ax.plot(m_df_region_total['Region'],m_df_region_total['total_vaccinations'],color='red',label='total')
plt.title('total vaccines')
plt.show()

In [ ]:
#plot heatmap to see the correlation between features
plt.subplots(figsize=(9, 9))
sns.heatmap(m_df.corr(), annot=True, square=True)
plt.show()

In [ ]:
# let's go deeply in the correlation
plt.subplots(figsize=(9, 9))
ax=sns.regplot(x=m_df['GDP (million in US$)'], y=m_df['total_vaccinations'])
#plt.ylim(0,)
ax.set_yscale('log')
ax.set_xscale('log')